In [1]:
import torch
import torch.nn as nn
from torchvision import transforms
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
from Dataset import Dataset
from models import Models



In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

test_dataset = Dataset(is_validation=True, device=device)

batch_size = 32
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)

In [19]:
criterion = nn.CrossEntropyLoss()

test_loss = 0.0
correct = 0
total = 0

with torch.no_grad():
    for images, labels in test_dataloader:
        images = images.to(device)
        labels = labels.to(device)

        # Forward pass
        outputs = model(images)
        #print(outputs.data.shape)
        #print(labels.shape)
        loss = criterion(outputs, labels)
        test_loss += loss.item()

        # Get predicted labels
        _, predicted = torch.max(outputs.data, 1)
        #print(_.shape)
        #print(predicted.shape)
        total += labels.size(0)
        correct += (predicted == labels[:, 0]).sum().item()

# Calculate average loss and accuracy
average_loss = test_loss / len(test_dataloader)
accuracy = correct / total * 100

print(f"Average Loss: {average_loss:.4f}")
print(f"Accuracy: {accuracy:.2f}%")

torch.Size([32, 2])
torch.Size([32, 2])
torch.Size([32])
torch.Size([32])
torch.Size([32, 2])
torch.Size([32, 2])
torch.Size([32])
torch.Size([32])
torch.Size([32, 2])
torch.Size([32, 2])
torch.Size([32])
torch.Size([32])
torch.Size([32, 2])
torch.Size([32, 2])
torch.Size([32])
torch.Size([32])
torch.Size([32, 2])
torch.Size([32, 2])
torch.Size([32])
torch.Size([32])


KeyboardInterrupt: 

In [13]:
torch.max(outputs, 1)#.data

torch.return_types.max(
values=tensor([ 0.2562,  0.1982,  0.2606,  0.0686,  0.1090, -0.0575,  0.2364, -0.0841,
         0.0614, -0.0201,  0.0827,  0.2294,  0.0981,  0.1201,  0.0082, -0.0043,
         0.1672,  0.1404, -0.0472,  0.2842,  0.0605,  0.1118,  0.2096,  0.1043,
         0.4236,  0.2330,  0.0787,  0.3049,  0.1617,  0.0758,  0.4172,  0.0199]),
indices=tensor([1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1]))

In [ ]:
image, label = test_dataset[0]
image_np = image.permute(1, 2, 0).numpy()

# Display the image
plt.imshow(image_np)
plt.title(f"Label: {label}")
plt.axis('off')
plt.show()
print(label)        # Label value

In [ ]:
from glob import glob
folder = f"{'val'}"
fake_imgs = [f'FakeManipulation-{i+1}/**/*.jpg' for i in range(5)]
fake_paths = [glob(f'{folder}/{img}', recursive=True) for img in fake_imgs]
real_imgs = [f'Real-{i+1}/**/*.jpg' for i in range(4)]
real_paths = [glob(f'{folder}/{img}', recursive=True) for img in real_imgs]

In [ ]:
fake_paths